# TNFR Periodic Table Atlas (Z=1..10)

This notebook builds element-like radial graphs using TNFR helpers, computes the Structural Field Tetrad (Φ_s, |∇φ|, K_φ, ξ_C), runs a sequential U6 (ΔΦ_s) check, and exports an HTML table mirroring the script.

In [ ]:
# Imports and setup
from tnfr.examples_utils import build_element_radial_graph, apply_synthetic_activation_sequence
from tnfr.physics.fields import (
    compute_structural_potential, compute_phase_gradient, compute_phase_curvature, estimate_coherence_length
)
from tnfr.operators.grammar import (
    warn_phase_gradient_telemetry, warn_phase_curvature_telemetry, warn_coherence_length_telemetry, validate_structural_potential_confinement
)
from tnfr.telemetry.constants import (
    PHASE_GRADIENT_THRESHOLD, PHASE_CURVATURE_ABS_THRESHOLD, STRUCTURAL_POTENTIAL_DELTA_THRESHOLD
)
from importlib import import_module
import os
import pandas as pd

periodic_mod = import_module('examples.periodic_table_atlas')  # reuse render_html and symbols

In [ ]:
# Analyze elements Z=1..10 and display a DataFrame
rows = []
for Z in range(1, 11):
    G = build_element_radial_graph(Z, seed=42)
    phi_before = compute_structural_potential(G)
    grad = compute_phase_gradient(G)
    kphi = compute_phase_curvature(G)
    xi_c = float(estimate_coherence_length(G))
    _, stats_g, msg_g, _ = warn_phase_gradient_telemetry(G, threshold=PHASE_GRADIENT_THRESHOLD)
    _, stats_k, msg_k, _ = warn_phase_curvature_telemetry(G, abs_threshold=PHASE_CURVATURE_ABS_THRESHOLD, multiscale_check=True, alpha_hint=2.76)
    _, stats_x, msg_x = warn_coherence_length_telemetry(G)
    apply_synthetic_activation_sequence(G, alpha=0.25, dnfr_factor=0.9)
    phi_after = compute_structural_potential(G)
    ok, drift, msg_u6 = validate_structural_potential_confinement(G, phi_before, phi_after, threshold=STRUCTURAL_POTENTIAL_DELTA_THRESHOLD, strict=False)
    total = max(1, len(G.nodes()))
    local_count = sum(
        1 for n in G.nodes()
        if abs(float(grad.get(n, 0.0))) < PHASE_GRADIENT_THRESHOLD and abs(float(kphi.get(n, 0.0))) < PHASE_CURVATURE_ABS_THRESHOLD
)
    rows.append({
        'Z': Z,
        'symbol': periodic_mod.SYMBOLS.get(Z, str(Z)),
        'xi_c': xi_c,
        'mean_grad': float(stats_g.get('mean_abs', 0.0)),
        'mean_kphi': float(stats_k.get('mean_abs', 0.0)),
        'mean_path_length': float(stats_x.get('mean_path_length', 0.0)),
        'local_frac': float(local_count) / float(total),
        'u6_ok': bool(ok),
        'u6_drift': float(drift),
    })
df = pd.DataFrame(rows)
df

In [ ]:
# Export HTML parity with the script
base = os.path.join(os.path.dirname(periodic_mod.__file__), 'output')
out_html = os.path.join(base, 'periodic_table_atlas.html')
periodic_mod.render_html(rows, out_path=out_html)
print('Wrote periodic table atlas to:', out_html)

In [ ]:
# Optional: JSONL/CSV exports to mirror the script outputs
out_jsonl = os.path.join(base, 'periodic_table_atlas.jsonl')
out_csv = os.path.join(base, 'periodic_table_atlas.csv')
# Reuse internal helpers if present; fallback to simple writers
try:
    periodic_mod._write_jsonl(rows, out_jsonl)
    periodic_mod._write_csv(rows, out_csv)
except Exception:
    import json
    os.makedirs(base, exist_ok=True)
    with open(out_jsonl, 'w', encoding='utf-8') as f:
        for r in rows:
            f.write(json.dumps(r) + "\n")
    import csv
    with open(out_csv, 'w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['Z','symbol','xi_c','mean_grad','mean_kphi','mean_path_length','local_frac','u6_ok','u6_drift'])
        writer.writeheader()
        for r in rows:
            writer.writerow({k: r.get(k) for k in writer.fieldnames})
print('Exported JSONL and CSV to:', base)

In [ ]:
# Summary by Signature (telemetry-only)
# This cell mirrors the HTML/CSV summary produced by examples/periodic_table_atlas.py
# It does not change operator dynamics; it only aggregates telemetry.
from collections import defaultdict
from statistics import mean
import pandas as pd

# Try to source rows from existing variables
rows_data = None
try:
    if isinstance(rows, list) and rows and isinstance(rows[0], dict):
        rows_data = rows
except NameError:
    rows_data = None

if rows_data is None:
    try:
        # Attempt to reconstruct from a DataFrame named df, if present
        if 'df' in globals():
            rows_data = [
                {
                    'signature': rec.get('signature', ''),
                    'u6_ok': bool(rec.get('u6_ok', False)),
                    'xi_c': float(rec.get('xi_c', 0.0)),
                    'mean_grad': float(rec.get('mean_grad', 0.0)),
                    'mean_kphi': float(rec.get('mean_kphi', 0.0)),
                    'local_frac': float(rec.get('local_frac', 0.0)),
                }
                for rec in df.to_dict(orient='records')
            ]
    except Exception as e:
        print('Could not reconstruct rows from df:', e)
        rows_data = None

if not rows_data:
    print('No telemetry rows found. Run the earlier cells that build rows/df first.')
else:
    groups = defaultdict(list)
    for r in rows_data:
        groups[str(r.get('signature', ''))].append(r)

    def safe_mean(vals):
        return float(mean(vals)) if vals else 0.0

    summary_records = []
    total = sum(len(v) for v in groups.values()) or 1
    dist_records = []

    for sig, items in groups.items():
        cnt = len(items)
        pass_rate = safe_mean([1.0 if it.get('u6_ok', False) else 0.0 for it in items])
        mean_xi = safe_mean([float(it.get('xi_c', 0.0)) for it in items])
        mean_g = safe_mean([float(it.get('mean_grad', 0.0)) for it in items])
        mean_k = safe_mean([float(it.get('mean_kphi', 0.0)) for it in items])
        mean_loc = safe_mean([float(it.get('local_frac', 0.0)) for it in items])
        summary_records.append({
            'signature': sig,
            'count': cnt,
            'pass_rate': pass_rate,
            'mean_xi_c': mean_xi,
            'mean_grad': mean_g,
            'mean_kphi': mean_k,
            'mean_local_frac': mean_loc,
        })
        dist_records.append({'signature': sig, 'count': cnt, 'pct': 100.0 * cnt / float(total)})

    summary_df = pd.DataFrame(summary_records).sort_values(by=['signature']).reset_index(drop=True)
    dist_df = pd.DataFrame(dist_records).sort_values(by=['signature']).reset_index(drop=True)

    display(summary_df)
    display(dist_df)


In [ ]:
# Save Summary by Signature to disk (parity with script outputs)
# Writes:
# - examples/output/periodic_table_atlas_by_signature.csv
# - examples/output/periodic_table_atlas_summary.json
import os, json
from pathlib import Path

# Reuse summary_df/dist_df if present; else try to rebuild via the previous logic
try:
    have_summary = 'summary_df' in globals() and 'dist_df' in globals()
except Exception:
    have_summary = False

if not have_summary:
    # Attempt minimal rebuild using the same approach as previous cell
    from collections import defaultdict
    from statistics import mean
    rows_data = None
    try:
        if isinstance(rows, list) and rows and isinstance(rows[0], dict):
            rows_data = rows
    except NameError:
        rows_data = None
    if rows_data is None and 'df' in globals():
        rows_data = [
            {
                'signature': rec.get('signature', ''),
                'u6_ok': bool(rec.get('u6_ok', False)),
                'xi_c': float(rec.get('xi_c', 0.0)),
                'mean_grad': float(rec.get('mean_grad', 0.0)),
                'mean_kphi': float(rec.get('mean_kphi', 0.0)),
                'local_frac': float(rec.get('local_frac', 0.0)),
            }
            for rec in df.to_dict(orient='records')
        ]
    if not rows_data:
        print('No telemetry rows found. Run the earlier cells first.')
    else:
        groups = defaultdict(list)
        for r in rows_data:
            groups[str(r.get('signature', ''))].append(r)
        def safe_mean(vals):
            return float(mean(vals)) if vals else 0.0
        summary_records = []
        for sig, items in groups.items():
            summary_records.append({
                'signature': sig,
                'count': len(items),
                'pass_rate': safe_mean([1.0 if it.get('u6_ok', False) else 0.0 for it in items]),
                'mean_xi_c': safe_mean([float(it.get('xi_c', 0.0)) for it in items]),
                'mean_grad': safe_mean([float(it.get('mean_grad', 0.0)) for it in items]),
                'mean_kphi': safe_mean([float(it.get('mean_kphi', 0.0)) for it in items]),
                'mean_local_frac': safe_mean([float(it.get('local_frac', 0.0)) for it in items]),
            })
        import pandas as pd
        summary_df = pd.DataFrame(summary_records)

if 'summary_df' not in globals():
    print('No summary_df available; nothing to write.')
else:
    # Locate repo root by searching for pyproject.toml
    def find_repo_root(start: Path) -> Path:
        for p in [start, *start.parents]:
            if (p / 'pyproject.toml').exists():
                return p
        return start
    repo_root = find_repo_root(Path.cwd())
    out_dir = repo_root / 'examples' / 'output'
    out_dir.mkdir(parents=True, exist_ok=True)

    groups_csv = out_dir / 'periodic_table_atlas_by_signature.csv'
    summary_json = out_dir / 'periodic_table_atlas_summary.json'

    # Write CSV
    summary_df.to_csv(groups_csv, index=False)

    # Write JSON as mapping signature -> metrics
    summary_map = {
        str(row['signature']): {
            'count': float(row['count']),
            'pass_rate': float(row['pass_rate']),
            'mean_xi_c': float(row['mean_xi_c']),
            'mean_grad': float(row['mean_grad']),
            'mean_kphi': float(row['mean_kphi']),
            'mean_local_frac': float(row['mean_local_frac']),
        }
        for _, row in summary_df.fillna(0.0).iterrows()
    }
    with open(summary_json, 'w', encoding='utf-8') as f:
        json.dump(summary_map, f, ensure_ascii=False, indent=2)

    print('Wrote:', groups_csv)
    print('Wrote:', summary_json)
